# Protótipo do Modelo Preditivo

### Importação de bibliotecas

In [97]:
import zipfile
import json
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Criando métodos

In [98]:
def ler_json(caminho_zip):
    """Metodo criado para ler um arquivo .json que esteja dentro de um .zip
    Estrutura do json deve ser chave -> elemento - > valor
    ou
    chave -> categoria -> elemento -> valor

    Args:
        caminho_zip (string): caminho onde está o arquivo .zip

    Returns:
        dataframe: o json sera achatado para retornar um dataframe correspondente,
        concatenando a categoria.elemento quando necessário
    """
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        nome_arquivo_json = zip_ref.namelist()[0]  # assumindo que só há um arquivo no zip
        with zip_ref.open(nome_arquivo_json) as file:
            dados = json.load(file)

    # Achatar os dados
    linhas = []
    for chave, conteudo in dados.items():
        linha = {"ID": chave}
        # Flatten (achatar) cada subnível com prefixo
        for secao, valores in conteudo.items():
            if isinstance(valores, dict):
                for subchave, subvalor in valores.items():
                    linha[f"{secao}.{subchave}"] = subvalor
            else:
                linha[secao] = valores
        linhas.append(linha)

    # Criar o DataFrame
    df = pd.DataFrame(linhas)
    return df


def ler_json_candidaturas(caminho_zip):
    """Similar ao metodo ler_json, porem, contemplando uma lista de elementos dentro
    de uma categoria

    Args:
        caminho_zip (string): caminho onde está o arquivo .zip

    Returns:
        dataframe: o json sera achatado para retornar um dataframe correspondente,
        concatenando a categoria.elemento quando necessário.
        Para o caso de uma lista de elementos, cada um vai gerar uma nova linha para a
        mesma chave inicial
    """
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        nome_arquivo_json = zip_ref.namelist()[0]  # assumindo que só há um arquivo no zip
        with zip_ref.open(nome_arquivo_json) as file:
            dados_candidatura = json.load(file)

    linhas_candidatura = []

    for codigo_vaga, conteudo in dados_candidatura.items():
        prospects = conteudo.get("prospects")

        # Verifica se há prospects e se é uma lista com itens
        if isinstance(prospects, list) and prospects:
            for prospect in prospects:
                linha_candidatura = {
                    'codigo_vaga': codigo_vaga,
                    'titulo_vaga': conteudo.get('titulo', ''),
                    'nome': prospect.get('nome', ''),
                    'codigo_candidato': prospect.get('codigo', ''),
                    'situacao_candidato': prospect.get('situacao_candidado', ''),
                    'data_candidatura': prospect.get('data_candidatura', ''),
                    'ultima_atualizacao': prospect.get('ultima_atualizacao', ''),
                    'comentario': prospect.get('comentario', ''),
                    'recrutador': prospect.get('recrutador', '')
                }
                linhas_candidatura.append(linha_candidatura)

    # Criar o DataFrame final
    df_candidaturas = pd.DataFrame(linhas_candidatura)
    return df_candidaturas

def exibir_head_df(nome_tabela, df):
    """Metodo para acompanhamento de dataframes

    Args:
        nome_tabela (string): nome a ser exibido no 'cabecalho'
        df (dataframe): dataframe a ser exibido
    """
    tabela = nome_tabela
    print('\n','-'*(len(tabela)+6))
    print('| ', tabela, ' |')
    print('-'*(len(tabela)+6))
    print(df.head(2))

### Carga e tratamento dos dados

In [99]:
#########################################################################################
# CARGA DOS DADOS
#########################################################################################

path_applicants = 'applicants.zip'
path_vagas = 'vagas.zip'
path_prospects = 'prospects.zip'

df_applicants = ler_json(path_applicants)
df_vagas      = ler_json(path_vagas)
df_prospects  = ler_json_candidaturas(path_prospects)

#########################################################################################
# TRATAMENTO DOS DADOS
#########################################################################################
#   PROSPECTS
#########################################################################################

# Removendo colunas que consideramos nao influenciar na analise
df_prospects.drop(columns=['nome', 'data_candidatura',
                           'recrutador', 'ultima_atualizacao',
                           'comentario', 'recrutador'], inplace=True)

#########################################################################################
#   VAGAS
#########################################################################################

# Removendo colunas que consideramos nao influenciar na analise
df_vagas.drop(columns=['informacoes_basicas.data_requicisao', 'informacoes_basicas.limite_esperado_para_contratacao',
                       'informacoes_basicas.cliente', 'informacoes_basicas.solicitante_cliente',
                       'informacoes_basicas.empresa_divisao', 'informacoes_basicas.requisitante',
                       'informacoes_basicas.analista_responsavel','informacoes_basicas.superior_imediato',
                       'informacoes_basicas.telefone', 'perfil_vaga.demais_observacoes',
                       'informacoes_basicas.data_inicial', 'informacoes_basicas.data_final',
                       'informacoes_basicas.nome_substituto','informacoes_basicas.nome',
                       'informacoes_basicas.origem_vaga', 'perfil_vaga.principais_atividades',
                       'beneficios.valor_compra_1', 'beneficios.valor_compra_2',
                       'perfil_vaga.habilidades_comportamentais_necessarias', 'beneficios.valor_venda',
                       'perfil_vaga.competencia_tecnicas_e_comportamentais', 'perfil_vaga.horario_trabalho'
                       ], inplace=True)

# Preenchendo nan com o "padrao" das ja utilizado em outras linhas e padronizando demais casos
df_vagas['perfil_vaga.equipamentos_necessarios'] = df_vagas['perfil_vaga.equipamentos_necessarios'].fillna('Nenhum -')
df_vagas['perfil_vaga.equipamentos_necessarios'] = df_vagas['perfil_vaga.equipamentos_necessarios'].str \
  .replace('Outro - Nenhum -', 'Nenhum -') \
  .replace('Outro -', 'Nenhum -') \
  .replace('', 'Nenhum -')

# Extrair os valores numéricos usando regex para ter a faixa etaria de forma mais clara
# Proposta de uso, no cadastro do candidato podemos realizar um filtro da idade dele pela idade ofertada pelas vagas?
df_vagas[['idade_minima', 'idade_maxima']] = df_vagas['perfil_vaga.faixa_etaria'].str.extract(r'De:\s*(\d+)\s*Até:\s*(\d+)')
df_vagas['idade_minima'] = df_vagas['idade_minima'].fillna(18).astype(int)
df_vagas['idade_maxima'] = df_vagas['idade_maxima'].fillna(60).astype(int)

#########################################################################################
#   APPLICANTS
#########################################################################################

# Removendo colunas que consideramos nao influenciar na analise
df_applicants.drop(columns=['infos_basicas.telefone_recado', 'infos_basicas.telefone',
                            'infos_basicas.inserido_por', 'infos_basicas.email',
                            'infos_basicas.sabendo_de_nos_por', 'infos_basicas.nome',
                            'infos_basicas.data_criacao', 'infos_basicas.data_atualizacao',
                            'informacoes_pessoais.nome', 'informacoes_pessoais.cpf',
                            'informacoes_pessoais.fonte_indicacao', 'informacoes_pessoais.email',
                            'informacoes_pessoais.email_secundario', 'informacoes_pessoais.data_nascimento',
                            'informacoes_pessoais.telefone_celular',
                            'informacoes_pessoais.telefone_recado', 'informacoes_pessoais.sexo',
                            'informacoes_pessoais.estado_civil', 'informacoes_pessoais.pcd',
                            'informacoes_pessoais.endereco', 'informacoes_pessoais.skype',
                            'informacoes_pessoais.url_linkedin', 'informacoes_pessoais.facebook',
                            'formacao_e_idiomas.instituicao_ensino_superior', 'informacoes_pessoais.download_cv',
                            'cargo_atual.nome_superior_imediato','cargo_atual.email_superior_imediato',
                            'cv_pt', 'cv_en', 'formacao_e_idiomas.outro_curso'], inplace=True)

# exibir_head_df('CANDIDATOS', df_applicants)
# exibir_head_df('VAGAS', df_vagas)
# exibir_head_df('CANDIDATURA', df_prospects)

### Análise exploratória e pré-processamento dos dados

#### Applicants

In [100]:
df_applicants.shape

(42482, 28)

In [101]:
df_applicants.head()

,ID,infos_basicas.objetivo_profissional,infos_basicas.local,infos_basicas.codigo_profissional,informacoes_pessoais.data_aceite,informacoes_profissionais.titulo_profissional,informacoes_profissionais.area_atuacao,informacoes_profissionais.conhecimentos_tecnicos,informacoes_profissionais.certificacoes,informacoes_profissionais.outras_certificacoes,informacoes_profissionais.remuneracao,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol,formacao_e_idiomas.outro_idioma,formacao_e_idiomas.cursos,formacao_e_idiomas.ano_conclusao,informacoes_profissionais.qualificacoes,informacoes_profissionais.experiencias,cargo_atual.id_ibrati,cargo_atual.email_corporativo,cargo_atual.cargo_atual,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade,cargo_atual.data_admissao,cargo_atual.data_ultima_promocao
0,31000,,,31000,Cadastro anterior ao registro de aceite,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,Analista Administrativo,"São Paulo, São Paulo",31001,Cadastro anterior ao registro de aceite,Analista Administrativo,Administrativa,,,,1900,,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,Administrativo | Financeiro,"São Paulo, São Paulo",31002,Cadastro anterior ao registro de aceite,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,Área administrativa,"São Paulo, São Paulo",31003,Cadastro anterior ao registro de aceite,Área administrativa,Administrativa,,,,"1100,00",,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,,31004,Cadastro anterior ao registro de aceite,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
df_applicants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 28 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   ID                                                42482 non-null  object
 1   infos_basicas.objetivo_profissional               42482 non-null  object
 2   infos_basicas.local                               42482 non-null  object
 3   infos_basicas.codigo_profissional                 42482 non-null  object
 4   informacoes_pessoais.data_aceite                  42482 non-null  object
 5   informacoes_profissionais.titulo_profissional     42482 non-null  object
 6   informacoes_profissionais.area_atuacao            42482 non-null  object
 7   informacoes_profissionais.conhecimentos_tecnicos  42482 non-null  object
 8   informacoes_profissionais.certificacoes           42482 non-null  object
 9   informacoes_profissionais.ou

In [103]:
print(df_applicants.isnull().sum())

ID                                                      0
infos_basicas.objetivo_profissional                     0
infos_basicas.local                                     0
infos_basicas.codigo_profissional                       0
informacoes_pessoais.data_aceite                        0
informacoes_profissionais.titulo_profissional           0
informacoes_profissionais.area_atuacao                  0
informacoes_profissionais.conhecimentos_tecnicos        0
informacoes_profissionais.certificacoes                 0
informacoes_profissionais.outras_certificacoes          0
informacoes_profissionais.remuneracao                   0
informacoes_profissionais.nivel_profissional            0
formacao_e_idiomas.nivel_academico                      0
formacao_e_idiomas.nivel_ingles                         0
formacao_e_idiomas.nivel_espanhol                       0
formacao_e_idiomas.outro_idioma                         0
formacao_e_idiomas.cursos                           35273
formacao_e_idi

In [104]:
df_applicants.nunique()

,0
ID,42482
infos_basicas.objetivo_profissional,8368
infos_basicas.local,721
infos_basicas.codigo_profissional,42482
informacoes_pessoais.data_aceite,11013
informacoes_profissionais.titulo_profissional,8368
informacoes_profissionais.area_atuacao,838
informacoes_profissionais.conhecimentos_tecnicos,3190
informacoes_profissionais.certificacoes,270
informacoes_profissionais.outras_certificacoes,884


In [105]:
cardinalidade_applicants = df_applicants.nunique()
colunas_baixa_card_applicants = cardinalidade_applicants[cardinalidade_applicants <= 30].index
len(colunas_baixa_card_applicants)

10

In [106]:
df_applicants_2 = df_applicants[['ID'] + list(colunas_baixa_card_applicants)]
df_applicants_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 11 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   ID                                            42482 non-null  object
 1   informacoes_profissionais.nivel_profissional  42482 non-null  object
 2   formacao_e_idiomas.nivel_academico            42482 non-null  object
 3   formacao_e_idiomas.nivel_ingles               42482 non-null  object
 4   formacao_e_idiomas.nivel_espanhol             42482 non-null  object
 5   informacoes_profissionais.qualificacoes       944 non-null    object
 6   informacoes_profissionais.experiencias        944 non-null    object
 7   cargo_atual.email_corporativo                 944 non-null    object
 8   cargo_atual.projeto_atual                     944 non-null    object
 9   cargo_atual.cliente                           944 non-null    object
 10

#### Prospects

In [107]:
df_prospects.shape

(53759, 4)

In [108]:
df_prospects.head()

,codigo_vaga,titulo_vaga,codigo_candidato,situacao_candidato
0,4530,CONSULTOR CONTROL M,25632,Encaminhado ao Requisitante
1,4530,CONSULTOR CONTROL M,25529,Encaminhado ao Requisitante
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25364,Contratado pela Decision
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,25360,Encaminhado ao Requisitante
4,4533,2021-2605708-Microfocus Application Life Cycle...,26338,Contratado pela Decision


In [109]:
df_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   codigo_vaga         53759 non-null  object
 1   titulo_vaga         53759 non-null  object
 2   codigo_candidato    53759 non-null  object
 3   situacao_candidato  53759 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB


In [110]:
print(df_prospects.isnull().sum())

codigo_vaga           0
titulo_vaga           0
codigo_candidato      0
situacao_candidato    0
dtype: int64


In [111]:
df_prospects.nunique()

,0
codigo_vaga,11279
titulo_vaga,9910
codigo_candidato,29405
situacao_candidato,21


In [112]:
cardinalidade_prospects = df_prospects.nunique()
colunas_baixa_card_prospects = cardinalidade_prospects[cardinalidade_prospects <= 30].index
len(colunas_baixa_card_prospects)

1

In [113]:
df_prospects_2 = df_prospects[['codigo_vaga'] + list(colunas_baixa_card_prospects)]
df_prospects_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   codigo_vaga         53759 non-null  object
 1   situacao_candidato  53759 non-null  object
dtypes: object(2)
memory usage: 840.1+ KB


#### Vagas

In [114]:
df_vagas.head()

,ID,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.tipo_contratacao,informacoes_basicas.prazo_contratacao,informacoes_basicas.objetivo_vaga,informacoes_basicas.prioridade_vaga,perfil_vaga.pais,perfil_vaga.estado,perfil_vaga.cidade,perfil_vaga.bairro,perfil_vaga.regiao,perfil_vaga.local_trabalho,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.faixa_etaria,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.areas_atuacao,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,idade_minima,idade_maxima
0,5185,Operation Lead -,Não,CLT Full,,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,,Nenhum -,18,60
1,5184,Consultor PP/QM Sênior,Não,CLT Full,,Contratação,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,,Nenhum -,18,60
2,5183,ANALISTA PL/JR C/ SQL,Não,CLT Full,,RFP,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,,Nenhum -,18,60
3,5182,Technical Architect - 11894809,Não,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Não,Notebook padrão -,18,60
4,5181,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,CLT Full,,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Sim,Nenhum -,18,60


In [115]:
df_vagas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     14081 non-null  object
 1   informacoes_basicas.titulo_vaga        14081 non-null  object
 2   informacoes_basicas.vaga_sap           14081 non-null  object
 3   informacoes_basicas.tipo_contratacao   14081 non-null  object
 4   informacoes_basicas.prazo_contratacao  14081 non-null  object
 5   informacoes_basicas.objetivo_vaga      14081 non-null  object
 6   informacoes_basicas.prioridade_vaga    14081 non-null  object
 7   perfil_vaga.pais                       14081 non-null  object
 8   perfil_vaga.estado                     14081 non-null  object
 9   perfil_vaga.cidade                     14081 non-null  object
 10  perfil_vaga.bairro                     14081 non-null  object
 11  perfil_vaga.reg

In [116]:
print(df_vagas.isnull().sum())

ID                                       0
informacoes_basicas.titulo_vaga          0
informacoes_basicas.vaga_sap             0
informacoes_basicas.tipo_contratacao     0
informacoes_basicas.prazo_contratacao    0
informacoes_basicas.objetivo_vaga        0
informacoes_basicas.prioridade_vaga      0
perfil_vaga.pais                         0
perfil_vaga.estado                       0
perfil_vaga.cidade                       0
perfil_vaga.bairro                       0
perfil_vaga.regiao                       0
perfil_vaga.local_trabalho               0
perfil_vaga.vaga_especifica_para_pcd     0
perfil_vaga.faixa_etaria                 0
perfil_vaga.nivel profissional           0
perfil_vaga.nivel_academico              0
perfil_vaga.nivel_ingles                 0
perfil_vaga.nivel_espanhol               0
perfil_vaga.outro_idioma                 0
perfil_vaga.areas_atuacao                0
perfil_vaga.viagens_requeridas           0
perfil_vaga.equipamentos_necessarios     0
idade_minim

In [117]:
df_vagas.nunique()

,0
ID,14081
informacoes_basicas.titulo_vaga,12452
informacoes_basicas.vaga_sap,2
informacoes_basicas.tipo_contratacao,40
informacoes_basicas.prazo_contratacao,3
informacoes_basicas.objetivo_vaga,6
informacoes_basicas.prioridade_vaga,4
perfil_vaga.pais,1
perfil_vaga.estado,27
perfil_vaga.cidade,167


In [118]:
cardinalidade_vagas = df_vagas.nunique()
colunas_baixa_card_vagas = cardinalidade_vagas[cardinalidade_vagas <= 30].index
len(colunas_baixa_card_vagas)

19

In [119]:
df_vagas_2 = df_vagas[['ID'] + list(colunas_baixa_card_vagas)]
df_vagas_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 20 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   ID                                     14081 non-null  object
 1   informacoes_basicas.vaga_sap           14081 non-null  object
 2   informacoes_basicas.prazo_contratacao  14081 non-null  object
 3   informacoes_basicas.objetivo_vaga      14081 non-null  object
 4   informacoes_basicas.prioridade_vaga    14081 non-null  object
 5   perfil_vaga.pais                       14081 non-null  object
 6   perfil_vaga.estado                     14081 non-null  object
 7   perfil_vaga.regiao                     14081 non-null  object
 8   perfil_vaga.local_trabalho             14081 non-null  object
 9   perfil_vaga.vaga_especifica_para_pcd   14081 non-null  object
 10  perfil_vaga.faixa_etaria               14081 non-null  object
 11  perfil_vaga.niv

#### Juntando os dataframes
Usando o df_prospects (interação do candidato com uma vaga) como ponte entre df_vagas (informações da vaga) e df_applicants (perfil do candidato)


In [120]:
df_vagas_2.set_index('ID', inplace=True)

df_merged = df_prospects_2.merge(df_vagas_2, left_on='codigo_vaga', right_index=True)
df_merged.head()

,codigo_vaga,situacao_candidato,informacoes_basicas.vaga_sap,informacoes_basicas.prazo_contratacao,informacoes_basicas.objetivo_vaga,informacoes_basicas.prioridade_vaga,perfil_vaga.pais,perfil_vaga.estado,perfil_vaga.regiao,perfil_vaga.local_trabalho,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.faixa_etaria,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,idade_minima,idade_maxima
0,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60
1,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60
2,4531,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60
3,4531,Encaminhado ao Requisitante,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60
4,4533,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Técnico,Fluente,,,Nenhum -,18,60


In [121]:
df_applicants_2.set_index('ID', inplace=True)

df_merged = df_merged.merge(df_applicants_2, left_on='codigo_vaga', right_index=True)
df_merged.head()

,codigo_vaga,situacao_candidato,informacoes_basicas.vaga_sap,informacoes_basicas.prazo_contratacao,informacoes_basicas.objetivo_vaga,informacoes_basicas.prioridade_vaga,perfil_vaga.pais,perfil_vaga.estado,perfil_vaga.regiao,perfil_vaga.local_trabalho,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.faixa_etaria,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,idade_minima,idade_maxima,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol,informacoes_profissionais.qualificacoes,informacoes_profissionais.experiencias,cargo_atual.email_corporativo,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade
0,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN
2,4531,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN
3,4531,Encaminhado ao Requisitante,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN
4,4533,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Técnico,Fluente,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN


#### Criando a variável alvo

In [122]:
df_merged['alvo'] = df_merged['situacao_candidato'].str.lower().str.contains('contratado').astype(int)
df_merged.head()

,codigo_vaga,situacao_candidato,informacoes_basicas.vaga_sap,informacoes_basicas.prazo_contratacao,informacoes_basicas.objetivo_vaga,informacoes_basicas.prioridade_vaga,perfil_vaga.pais,perfil_vaga.estado,perfil_vaga.regiao,perfil_vaga.local_trabalho,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.faixa_etaria,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,idade_minima,idade_maxima,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol,informacoes_profissionais.qualificacoes,informacoes_profissionais.experiencias,cargo_atual.email_corporativo,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade,alvo
0,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN,0
1,4530,Encaminhado ao Requisitante,Não,,,,Brasil,Rio de Janeiro,,2000,Não,De: Até:,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4531,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4531,Encaminhado ao Requisitante,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Nenhum,Nenhum,,,Notebook padrão -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4533,Contratado pela Decision,Não,,Contratação,Média: Média complexidade 6 a 10 dias,Brasil,São Paulo,,2000,Não,De: Até:,Sênior,Ensino Médio Completo,Técnico,Fluente,,,Nenhum -,18,60,,,,,NaN,NaN,NaN,NaN,NaN,NaN,1


In [123]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53712 entries, 0 to 53758
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   codigo_vaga                                   53712 non-null  object
 1   situacao_candidato                            53712 non-null  object
 2   informacoes_basicas.vaga_sap                  53712 non-null  object
 3   informacoes_basicas.prazo_contratacao         53712 non-null  object
 4   informacoes_basicas.objetivo_vaga             53712 non-null  object
 5   informacoes_basicas.prioridade_vaga           53712 non-null  object
 6   perfil_vaga.pais                              53712 non-null  object
 7   perfil_vaga.estado                            53712 non-null  object
 8   perfil_vaga.regiao                            53712 non-null  object
 9   perfil_vaga.local_trabalho                    53712 non-null  object
 10  per

### Treinamento com Random Forest

#### Pipeline completo

In [124]:
X = df_merged.drop(columns=['alvo'])
y = df_merged['alvo']

In [125]:
colunas_categoricas = X.select_dtypes(include='object').columns.tolist()
colunas_numericas = X.select_dtypes(include='number').columns.tolist()

In [126]:
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='desconhecido')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessador = ColumnTransformer(transformers=[
    ('cat', cat_pipeline, colunas_categoricas),
    ('num', num_pipeline, colunas_numericas)
])

#### Criando e treinando o modelo

In [127]:
modelo = Pipeline(steps=[
    ('preprocessamento', preprocessador),
    ('classificador', RandomForestClassifier(n_estimators=100, random_state=42))
])

In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelo.fit(X_train, y_train)
y_pred = modelo.predict(X_test)

#### Avaliando o modelo

In [129]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10135
           1       1.00      0.98      0.99       608

    accuracy                           1.00     10743
   macro avg       1.00      0.99      0.99     10743
weighted avg       1.00      1.00      1.00     10743



In [130]:
# Testar GridSearchCV pra ajustar os hiperparâmetros da floresta